# CSV + Pandas Dataframe

In [10]:
!pip install -qU langchain langchain-openai langchain_community langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 1.6 MB/s eta 0:00:00


In [7]:
import os

os.environ["OPENAI_API_KEY"] = 'sk-***********************************************'

In [2]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='/content/mlb_teams_2012.csv')
data = loader.load()

In [5]:
data[0].page_content

'Team: Nationals\n"Payroll (millions)": 81.34\n"Wins": 98'

In [6]:
data[0].metadata

{'source': '/content/mlb_teams_2012.csv', 'row': 0}

## Usecases

### create_csv_agent

In [63]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain_openai import ChatOpenAI, OpenAI

agent = create_csv_agent(
    ChatOpenAI(temperature=0, model="gpt-4"),
    "/content/mlb_teams_2012.csv",
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [61]:
agent.run("how many rows are there?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.shape[0]'}`


30There are 30 rows in the dataframe.

> Finished chain.


'There are 30 rows in the dataframe.'

In [64]:
agent.run("Which team has the most wins?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Wins'] == df['Wins'].max()]['Team'].values[0]"}`


NationalsThe team with the most wins is the Nationals.

> Finished chain.


'The team with the most wins is the Nationals.'

## Customizing the CSV parsing and loading

In [65]:
loader = CSVLoader(file_path='/content/mlb_teams_2012.csv', csv_args={
    'delimiter': ',',
    'quotechar': '"',
    'fieldnames': ['MLB Team', 'Payroll in millions', 'Wins']
})

data = loader.load()

In [66]:
data[0].page_content

'MLB Team: Team\nPayroll in millions: Payroll\nWins: Wins'

In [67]:
data[0].metadata

{'source': '/content/mlb_teams_2012.csv', 'row': 0}

## Specify a column to identify the document source

In [68]:
loader = CSVLoader(file_path='/content/mlb_teams_2012.csv', source_column="Team")

data = loader.load()


In [69]:
print(data)

[Document(page_content='Team: Nationals\nPayroll: 81.34\nWins: 98', metadata={'source': 'Nationals', 'row': 0}), Document(page_content='Team: Reds\nPayroll: 82.2\nWins: 97', metadata={'source': 'Reds', 'row': 1}), Document(page_content='Team: Yankees\nPayroll: 197.96\nWins: 95', metadata={'source': 'Yankees', 'row': 2}), Document(page_content='Team: Giants\nPayroll: 117.62\nWins: 94', metadata={'source': 'Giants', 'row': 3}), Document(page_content='Team: Braves\nPayroll: 83.31\nWins: 94', metadata={'source': 'Braves', 'row': 4}), Document(page_content='Team: Athletics\nPayroll: 55.37\nWins: 94', metadata={'source': 'Athletics', 'row': 5}), Document(page_content='Team: Rangers\nPayroll: 120.51\nWins: 93', metadata={'source': 'Rangers', 'row': 6}), Document(page_content='Team: Orioles\nPayroll: 81.43\nWins: 93', metadata={'source': 'Orioles', 'row': 7}), Document(page_content='Team: Rays\nPayroll: 64.17\nWins: 90', metadata={'source': 'Rays', 'row': 8}), Document(page_content='Team: Ange

In [76]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain_openai import ChatOpenAI, OpenAI

agent = create_csv_agent(
    ChatOpenAI(temperature=0, model="gpt-4"),
    ["/content/mlb_teams_2012.csv","/content/mlb_players.csv"],
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [77]:
agent.run("플레이어 중 가장 키가 큰 선수는 누구에요?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df2[df2['Height(inches)'] == df2['Height(inches)'].max()]['Name']"}`


KeyError: 'Height(inches)'
Invoking: `python_repl_ast` with `{'query': 'df2.columns'}`
responded: 죄송합니다. 데이터 프레임에서 'Height(inches)' 열을 찾을 수 없습니다. 데이터 프레임의 열 이름을 확인하겠습니다.

Index(['Name', ' Team', ' Position', ' Height(inches)', ' Weight(lbs)',
       ' Age'],
      dtype='object')
Invoking: `python_repl_ast` with `{'query': "df2[df2[' Height(inches)'] == df2[' Height(inches)'].max()]['Name']"}`
responded: 데이터 프레임의 열 이름을 확인해 보니, 'Height(inches)' 앞에 공백이 있습니다. 이를 고려하여 다시 질문에 대한 답변을 찾아보겠습니다.

928    Jon Rauch
Name: Name, dtype: object가장 키가 큰 선수는 Jon Rauch입니다.

> Finished chain.


'가장 키가 큰 선수는 Jon Rauch입니다.'

# Pandas Dataframe

In [70]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

import pandas as pd
from langchain_openai import OpenAI

df = pd.read_csv("/content/mlb_teams_2012.csv")

agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-4"),
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [58]:
agent.run("how many rows are there?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.shape[0]'}`


30There are 30 rows in the dataframe.

> Finished chain.


'There are 30 rows in the dataframe.'

In [71]:
agent.run("Which team has the most wins?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Wins'] == df['Wins'].max()]['Team'].values[0]"}`


NationalsThe team with the most wins is the Nationals.

> Finished chain.


'The team with the most wins is the Nationals.'

In [72]:
agent.run("whats the square root of the average wins?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['Wins'].mean()"}`


81.0
Invoking: `python_repl_ast` with `{'query': 'import math\nmath.sqrt(81)'}`


9.0The square root of the average wins is 9.0.

> Finished chain.


'The square root of the average wins is 9.0.'

## Multi DataFrame Example

In [73]:
df1 = df.copy()
df1["Wins"] = df1["Wins"].fillna(df1["Wins"].mean())

In [75]:
agent = create_pandas_dataframe_agent(OpenAI(temperature=0), [df, df1], verbose=True)
agent.run("how many rows in the wins column are different?")



> Entering new AgentExecutor chain...
Thought: I need to compare the wins column in both dataframes and count the number of rows that are different.
Action: python_repl_ast
Action Input: len(df1[df1['Wins'] != df2['Wins']])00 rows are different in the wins column.
Final Answer: 0

> Finished chain.


'0'